<a href="https://colab.research.google.com/github/oussama-jarroud/fraude-detection-ML/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit
!pip install pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 109.6 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import joblib
import numpy as np

st.set_page_config(page_title="Fraud Detector Demo", layout="wide")

@st.cache_data
def load_artifacts():
    model = joblib.load("model_xgb.joblib")
    scaler = joblib.load("scaler_amount.joblib")
    features_order = joblib.load("features_order.joblib")
    return model, scaler, features_order

model, scaler, features_order = load_artifacts()

st.title("Détecteur de Fraude — Demo")
st.write("Tester une transaction unique (entrée manuelle) ou uploader un fichier CSV (batch).")

col1, col2 = st.columns(2)

with col1:
    st.header("1) Entrée manuelle")
    st.write("Remplis les valeurs pour chaque feature puis clique sur *Prédire (manuelle)*.")
    inputs = {}
    for feat in features_order:
        # par défaut 0.0 ; tu peux ajuster format selon feature
        inputs[feat] = st.number_input(label=feat, value=0.0, format="%.6f", step=0.1)

    if st.button("Prédire (manuelle)"):
        x = pd.DataFrame([inputs], columns=features_order)
        # Si tu as appliqué log1p ou Hour lors de l'entraînement, il faut le reproduire ici.
        # Exemple si tu as 'Hour' dépendant de 'Time', gérer ici (décommente si nécessaire):
        # if 'Hour' in features_order and 'Time' in x.columns:
        #     x['Hour'] = ((x['Time'] // 3600) % 24).astype(int)

        if 'Amount' in features_order:
            x['Amount'] = scaler.transform(x[['Amount']])
        proba = model.predict_proba(x)[:,1][0]
        pred = int(model.predict(x)[0])
        st.metric("Probabilité de fraude", f"{proba:.4f}")
        st.write("Prediction:", "🚨 **FRAUDE**" if pred==1 else "✅ Normal")

with col2:
    st.header("2) Batch : upload CSV")
    st.write("Le CSV doit contenir les colonnes suivantes (mêmes noms) :")
    st.write(features_order)
    uploaded_file = st.file_uploader("Uploader un CSV", type=["csv"])
    if uploaded_file is not None:
        df = pd.read_csv(uploaded_file)
        st.write("Aperçu upload :")
        st.dataframe(df.head())
        missing = [c for c in features_order if c not in df.columns]
        if missing:
            st.error(f"Colonnes manquantes: {missing}")
        else:
            X = df[features_order].copy()
            if 'Amount' in features_order:
                X['Amount'] = scaler.transform(X[['Amount']])
            df['fraud_proba'] = model.predict_proba(X)[:,1]
            df['pred_label'] = model.predict(X)
            st.write("Résultats (après prédiction) :")
            st.dataframe(df.head(20))
            csv = df.to_csv(index=False).encode('utf-8')
            st.download_button("Télécharger résultats CSV", data=csv, file_name='predictions.csv', mime='text/csv')

st.markdown("---")
st.write("Remarque : Si tu as fait des transformations (log1p(Amount), création Hour...) lors de l'entraînement, reproduis-les exactement ici avant la prédiction.")


Writing app.py


In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("Y8554UCH6T")


In [ ]:
!ngrok config add-authtoken Y8554UCH6T
!streamlit run app.py & npx localtunnel --port 8501



Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
⠙

⠹⠸⠼⠴⠦
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.106.199.91:8501

⠧⠇⠏⠋your url is: https://cool-lies-kick.loca.lt
